In [1]:
import sys

from sqlalchemy.dialects import postgresql
from sqlalchemy.schema import CreateTable

sys.path.append("../dags")

import os

os.environ["TABLE_PREFIX"] = "SLVK"

In [2]:
from modules.models import DeclarativeBase

In [3]:
for i, (name, cls) in enumerate(DeclarativeBase.registry._class_registry.items()):
    if name.startswith("_"):
        continue
    print("=" * 100)
    print(name, cls.__tablename__)
    print("\n".join(map(str, cls.__table__.columns)))

    if not cls.__tablename__.startswith("SLVK"):
        raise

    with open("../main.ddl.sql", "w" if i == 0 else "a") as fout:
        print(f"-- {name} -- ", file=fout)
        print(CreateTable(cls.__table__, if_not_exists=True).compile(dialect=postgresql.dialect()), file=fout)
        # print("", file=fout)

StagingAccount SLVK_stg_accounts
SLVK_stg_accounts.account
SLVK_stg_accounts.valid_to
SLVK_stg_accounts.client
DimAccount SLVK_dwh_dim_accounts
SLVK_dwh_dim_accounts.account
SLVK_dwh_dim_accounts.valid_to
SLVK_dwh_dim_accounts.client
SLVK_dwh_dim_accounts.create_dt
SLVK_dwh_dim_accounts.update_dt
DimAccountHist SLVK_dwh_dim_accounts_hist
SLVK_dwh_dim_accounts_hist.account
SLVK_dwh_dim_accounts_hist.valid_to
SLVK_dwh_dim_accounts_hist.client
SLVK_dwh_dim_accounts_hist.effective_from
SLVK_dwh_dim_accounts_hist.effective_to
SLVK_dwh_dim_accounts_hist.deleted_flg
StagingPassportBlacklist SLVK_stg_blacklist
SLVK_stg_blacklist.date
SLVK_stg_blacklist.passport
FactPassportBlacklist SLVK_dwh_fact_passport_blacklist
SLVK_dwh_fact_passport_blacklist.id
SLVK_dwh_fact_passport_blacklist.date
SLVK_dwh_fact_passport_blacklist.passport
StagingCard SLVK_stg_cards
SLVK_stg_cards.card_num
SLVK_stg_cards.account
DimCard SLVK_dwh_dim_cards
SLVK_dwh_dim_cards.card_num
SLVK_dwh_dim_cards.account
SLVK_dwh_di

In [4]:
import datetime

from modules.data.build_report import generate_fraud_report
from modules.data.dim_hist_load import load_to_dim_hist
from modules.data.dim_load import load_tables_to_dim
from modules.data.fact_load import load_fact_data
from modules.data.staging_load import load_data_into_staging
from modules.db.utils import engine
from modules.models import create_tables

for date in [datetime.date(2021, 3, 1), datetime.date(2021, 3, 2), datetime.date(2021, 3, 3)]:
    create_tables(engine=engine)
    load_data_into_staging(date=date)
    load_tables_to_dim()
    load_to_dim_hist()
    load_fact_data()
    generate_fraud_report(date=date)

INFO:root:start loading source: DimSource(path='info.clients', source_type='table', target_table=<class 'modules.models.client.StagingClient'>, validator=<class 'modules.models.client.ClientModel'>, dim_table=<class 'modules.models.client.DimClient'>, dim_hist_table=<class 'modules.models.client.DimClientHist'>)
INFO:root:loaded data from: info.clients
INFO:root:validate data with <class 'modules.models.client.ClientModel'>
INFO:root:truncate target table before loading: <class 'modules.models.client.StagingClient'>
INFO:root:loaded 50 records into db
INFO:root:start loading source: DimSource(path='../data/terminals_{today}.xlsx', source_type='excel', target_table=<class 'modules.models.terminal.StagingTerminal'>, validator=<class 'modules.models.terminal.TerminalModel'>, dim_table=<class 'modules.models.terminal.DimTerminal'>, dim_hist_table=<class 'modules.models.terminal.DimTerminalHist'>)
INFO:root:loaded data from: ../data/terminals_01032021.xlsx
INFO:root:validate data with <clas

In [5]:
from modules.data.build_report import generate_fraud_report
from modules.data.dim_hist_load import load_to_dim_hist
from modules.data.dim_load import load_tables_to_dim
from modules.data.fact_load import load_fact_data
from modules.data.staging_load import load_data_into_staging
from modules.db.utils import engine
from modules.models import create_tables

In [6]:
from modules.data.source import DEFAULT_SOURCE_LIST

In [7]:
source = DEFAULT_SOURCE_LIST[0]

In [16]:
from sqlalchemy.inspection import inspect

for source in DEFAULT_SOURCE_LIST:
    # print(source)
    inspect(source.target_table).primary_key
    print(
        set(map(lambda x: x.name, source.target_table.__mapper__.columns))
        ^ set(map(lambda x: x.name, source.dim_table.__mapper__.columns))
    )

{'update_dt', 'create_dt'}
{'update_dt', 'create_dt'}
{'update_dt', 'create_dt'}
{'update_dt', 'create_dt'}


AttributeError: 'FactSource' object has no attribute 'dim_table'

In [24]:
list(inspect(source.target_table).columns)

[Column('id', String(), table=<SLVK_stg_transactions>, primary_key=True, nullable=False),
 Column('date', DateTime(), table=<SLVK_stg_transactions>, nullable=False),
 Column('amount', Float(), table=<SLVK_stg_transactions>, nullable=False),
 Column('card_num', String(), table=<SLVK_stg_transactions>, nullable=False),
 Column('oper_type', String(), table=<SLVK_stg_transactions>, nullable=False),
 Column('oper_result', String(), table=<SLVK_stg_transactions>, nullable=False),
 Column('terminal', String(), table=<SLVK_stg_transactions>, nullable=False)]

In [21]:
source.target_table.id

In [22]:
source.target_table["id"]

TypeError: <class 'sqlalchemy.orm.decl_api.DeclarativeMeta'> is not a generic class

In [5]:
import sys

sys.path.append("../dags/")
import datetime

from modules.db.utils import Session
from modules.models import (
    DimAccount,
    DimCard,
    DimClient,
    DimTerminal,
    FactTransaction,
)
from sqlalchemy import Date, and_, cast, func, or_
from sqlalchemy.orm import Session

session = Session()

subquery = (
    session.query(
        FactTransaction.date.label("transaction_date"),
        DimClient.passport_num.label("passport"),
        DimClient.last_name,
        DimClient.first_name,
        DimClient.patronymic,
        DimClient.phone,
        FactTransaction.terminal,
        DimTerminal.city,
        func.lag(DimTerminal.city)
        .over(partition_by=DimClient.client_id, order_by=FactTransaction.date)
        .label("previous_city"),
        func.lag(FactTransaction.date)
        .over(partition_by=DimClient.client_id, order_by=FactTransaction.date)
        .label("previous_dttm"),
        func.lead(DimTerminal.city)
        .over(partition_by=DimClient.client_id, order_by=FactTransaction.date)
        .label("next_city"),
        func.lead(FactTransaction.date)
        .over(partition_by=DimClient.client_id, order_by=FactTransaction.date)
        .label("next_dttm"),
    )
    .select_from(FactTransaction)
    .join(DimCard, FactTransaction.card_num == DimCard.card_num)
    .join(DimAccount, DimCard.account == DimAccount.account)
    .join(DimClient, DimAccount.client == DimClient.client_id)
    .join(DimTerminal, FactTransaction.terminal == DimTerminal.id)
    .subquery()
)

query = (
    session.query(
        subquery.c.transaction_date,
        subquery.c.passport,
        subquery.c.last_name,
        subquery.c.first_name,
        subquery.c.patronymic,
        subquery.c.phone,
        subquery.c.terminal,
    )
    .filter(
        or_(
            and_(
                subquery.c.city != subquery.c.previous_city,
                subquery.c.transaction_date - subquery.c.previous_dttm < datetime.timedelta(hours=1),
            ),
            and_(
                subquery.c.city != subquery.c.next_city,
                subquery.c.next_dttm - subquery.c.transaction_date < datetime.timedelta(hours=1),
            ),
        ),
    )
    .filter(cast(FactTransaction.date, Date) == date)
)

In [6]:
str(datetime.timedelta(hours=1))

'1:00:00'

In [7]:
from sqlalchemy.dialects import postgresql

print(query.statement.compile(dialect=postgresql.dialect()))

SELECT anon_1.transaction_date, anon_1.passport, anon_1.last_name, anon_1.first_name, anon_1.patronymic, anon_1.phone, anon_1.terminal 
FROM (SELECT public."SLVK_dwh_fact_transactions".date AS transaction_date, public."SLVK_dwh_dim_clients".passport_num AS passport, public."SLVK_dwh_dim_clients".last_name AS last_name, public."SLVK_dwh_dim_clients".first_name AS first_name, public."SLVK_dwh_dim_clients".patronymic AS patronymic, public."SLVK_dwh_dim_clients".phone AS phone, public."SLVK_dwh_fact_transactions".terminal AS terminal, public."SLVK_dwh_dim_terminals".city AS city, lag(public."SLVK_dwh_dim_terminals".city) OVER (PARTITION BY public."SLVK_dwh_dim_clients".client_id ORDER BY public."SLVK_dwh_fact_transactions".date) AS previous_city, lag(public."SLVK_dwh_fact_transactions".date) OVER (PARTITION BY public."SLVK_dwh_dim_clients".client_id ORDER BY public."SLVK_dwh_fact_transactions".date) AS previous_dttm, lead(public."SLVK_dwh_dim_terminals".city) OVER (PARTITION BY public."SLV

In [19]:
query.all()

OperationalError: (psycopg2.OperationalError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: SELECT public."SLVK_dwh_fact_transactions".date AS transaction_date, public."SLVK_dwh_dim_clients".passport_num AS passport, public."SLVK_dwh_dim_clients".last_name AS "public_SLVK_dwh_dim_clients_last_name", public."SLVK_dwh_dim_clients".first_name AS "public_SLVK_dwh_dim_clients_first_name", public."SLVK_dwh_dim_clients".patronymic AS "public_SLVK_dwh_dim_clients_patronymic", public."SLVK_dwh_dim_clients".phone AS "public_SLVK_dwh_dim_clients_phone", public."SLVK_dwh_fact_transactions".terminal AS "public_SLVK_dwh_fact_transactions_terminal" 
FROM public."SLVK_dwh_fact_transactions" JOIN public."SLVK_dwh_dim_cards" ON public."SLVK_dwh_fact_transactions".card_num = public."SLVK_dwh_dim_cards".card_num JOIN public."SLVK_dwh_dim_accounts" ON public."SLVK_dwh_dim_cards".account = public."SLVK_dwh_dim_accounts".account JOIN public."SLVK_dwh_di